# Tirage aléatoire de stems

- à partir du fichier Stems.yaml
- vers le fichiers randomStems.yaml

In [1]:
# -*- coding: utf8 -*-
#########################IMPORTS############################################
from os.path import expanduser
import codecs, optparse
import re
import random as rd
import sys,os,time
import string
import yaml, YamlDuplicates
import ParFuMor as PFM
from ParFuMor import *
import pickle

In [2]:
from string import translate, maketrans

In [3]:
home = expanduser("~")
# repertoire=home+"/ownCloud/Cours/Bordeaux/L1-UE4-Morphologie/00-Kanonik/"+annee
repertoire=home+"/Github/Kalaba/Kalaba2/Kanonik/"
repertoire=home+"/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/00-ProjetKalaba/23-K5/"

with open(repertoire+"Stems.yaml", 'r') as stream:
    nullStems=yaml.load(stream,Loader=yaml.Loader)
    
noStems=False # pb avec True et les catégories intermédiaire
prevStems=nullStems.copy()
setRadicals=set()

In [4]:
nomsPropres=["AgathoS","Clemencia","Elphaba","Freja","Hoderi","Kaleb","Mahira","Violette","Nabil","Katisha","Nicole"]
nomsPropresP=["agaTos","klemensja","elfaba","freZa","oderi","kaleb","maira","vjolet","nabil","katiSa","nikol"]

In [5]:
with open(repertoire+"Phonology.yaml", 'r') as stream:
    phonology=yaml.load(stream,Loader=yaml.Loader)

In [6]:
obstrusivesSourdes=u"ptkfTsS"
obstrusivesVoisees=u"bdgvDzZ"

voisement=maketrans(obstrusivesSourdes,obstrusivesVoisees)
devoisement=maketrans(obstrusivesVoisees,obstrusivesSourdes)


consonnes=phonology["consonnes"]
voyelles=phonology["voyelles"]

In [7]:
from numpy.random import choice

consonnesFreq=[10,10,10,10,10,10,8,8,6,7,7,5,7,7,7,5,7,8,8,3,3]
consonnesTot=sum(consonnesFreq)
consonnesProb=[float(k)/consonnesTot for k in consonnesFreq]


In [8]:
for k,v in enumerate(consonnes):
    print k,v,consonnesFreq[k]

0 p 10
1 t 10
2 k 10
3 b 10
4 d 10
5 g 10
6 m 8
7 n 8
8 N 6
9 f 7
10 s 7
11 T 5
12 S 7
13 v 7
14 z 7
15 D 5
16 Z 7
17 r 8
18 l 8
19 j 3
20 w 3


In [9]:
def randomCVCVC():
    result=""
    result+=choice(list(consonnes), p=consonnesProb) #from numpy
    result+=rd.choice(voyelles)                      #from random
    result+=choice(list(consonnes), p=consonnesProb)
    result+=rd.choice(voyelles)
    result+=choice(list(consonnes), p=consonnesProb)
    if result not in setRadicals:
        return result
    else:
        return randomCVCVC()

In [10]:
if noStems:
    for cat in nullStems:
        prevStems[cat]={}
        for lex in nullStems[cat]:
            rdRadical=randomCVCVC()
            prevStems[cat][rdRadical]=nullStems[cat][lex]
            

In [11]:
def getLeaves(stemStructure):
    formes={}
    for element in stemStructure:
        if type(stemStructure[element])==list:
            formesCle=", ".join(stemStructure[element])
            formes[formesCle]=element
        else:
            formes.update(getLeaves(stemStructure[element]))
    return formes

In [12]:
prevLexique=getLeaves(prevStems)
nextLexique=getLeaves(prevStems)

In [13]:
def replace_keys(old_dict, key_dict):
    new_dict = { }
    for key in old_dict.keys():
        new_key = key_dict.get(key, key)
        if isinstance(old_dict[key], dict):
            new_dict[new_key] = replace_keys(old_dict[key], key_dict)
        else:
            new_dict[new_key] = old_dict[key]
    return new_dict


In [14]:
def devoiserFinale(radical):
    if radical[-1] in obstrusivesVoisees:
        return radical.translate(devoisement)
    else:
        return None

def voiserVCV(radical):
    result=list(radical)
    cibles=[]
    for p,l in enumerate(radical[1:-1]):
        if l in obstrusivesSourdes and radical[p] in voyelles and radical[p+2] in voyelles:
            cibles.append(p+1)
    if cibles:
        cible=rd.choice(cibles)
        result[cible]=result[cible].translate(voisement)
        return "".join(result)
    else:
        return None

def metathese(radical):
    result=list(radical)
    cibles=[]
    for p,l in enumerate(radical):
        if l in consonnes:
            cibles.append(p)
    if len(cibles)>1 and rd.randint(1,3)==1:
        cible1=rd.choice(cibles[:-1])
        cible2=cibles[cibles.index(cible1)+1]
        result[cible1]=radical[cible2]
        result[cible2]=radical[cible1]
        result="".join(result)
        if result!=radical:
            return result
        else:
            return None
    else:
        return None

    
def apophonie(radical):
    result=list(radical)
    cibles=[]
    for p,l in enumerate(radical):
        if l in "iae":
            cibles.append(p)
    if cibles:
        cible=rd.choice(cibles)
        result[cible]=phonology["apophonies"][result[cible]]
        return "".join(result)
    else:
        return None

def changerLongueur(radical):
    result=list(radical)
    if len(radical)==5:
        if radical[0] in consonnes:
            if radical[-1] in consonnes:
                if rd.randint(1,5)==1:
                    result.append(rd.choice(voyelles))
                    return "".join(result)
                else:
                    return None
            else:
                if rd.randint(1,5)==1:
                    result.append(rd.choice(consonnes))
                    return "".join(result)
                else:
                    return None
        else:
            if rd.randint(1,3)==1:
                result.insert(0,rd.choice(consonnes))
                return "".join(result)
            else:
                return None
    elif len(radical)>5:
        if rd.randint(1,3)==1:
            if radical[0] in consonnes:
                result=result[:-1]
            else:
                result=result[1:]
            return "".join(result)
        else:
            return None
    else:
        if rd.randint(1,3)==1:
            if radical[0] in voyelles:
                result.insert(0,rd.choice(consonnes))
            elif radical[-1] in voyelles:
                result.append(rd.choice(consonnes))
            else:
                result.append(rd.choice(voyelles))
            return "".join(result)
        else:
            return None
        
            

In [15]:
operations=[devoiserFinale,devoiserFinale,voiserVCV,voiserVCV,changerLongueur,changerLongueur,apophonie,metathese]

In [16]:
def randomRadical(radical):
    result=""
    for letter in radical:
        if letter in voyelles:
            result+=rd.choice(voyelles)
        else:
            result+=choice(list(consonnes), p=consonnesProb)
    print "random",result
    return result

def modifierRadical(radical,dist=[0,1,2,2,3,3,3,4,5]):
    nOperations=rd.choice(dist)
    while nOperations>0:
        operation=rd.choice(operations)
        print nOperations,operation
        result=operation(radical)
        if result:
            radical=result
            nOperations-=1
    print "modif",radical
    return radical

In [17]:
modifierRadical(randomRadical(u"kaleb"),dist=[0,0,0,1,1,2])

random dakat
modif dakat


'dakat'

In [18]:
prevLexique

{'Agathos, AgathoS': 'agaTos',
 'Clemencia, Clemencias': 'klemensja',
 'DEF, le, la, les, l': 'k',
 'DEM, ce, cet, cette, ces': 'b',
 'ENTRE, entrE': 'Sik',
 'Elphaba, Elphabas': 'elfaba',
 'Freja, Frejas': 'freZa',
 'Hoderi, Hoderis': 'oderi',
 'IND, un, une, des, dE': 'l',
 'Kaleb, Kalebs': 'kaleb',
 'Katisha, Katishas': 'katiSa',
 'Mahira, Mahiras': 'maira',
 'Nabil, Nabils': 'nabil',
 'Nicole, Nicoles': 'nikol',
 'PRO, il, elle, ils, elles, lui, eux': 'Zag',
 'Violette, Violettes': 'vjolet',
 'accueillir, accueille, accueillent, accueillait, accueillaient, accueillit, accueillirent': 'TotuS',
 u'acheter, ach\xe8te, ach\xe8tent, achetait, achetaient': 'Tozem',
 u'aller, va, vont, allait, allaient, alla, all\xe8rent': 'dovom',
 u'allumer, allume, allument, allumait, allumaient, alluma, allum\xe8rent': 'Sulowi',
 u'apporter, apporte, apportent, apportait, apportaient, apporta, apport\xe8rent': 'Nipot',
 'approcher, approche, approchent, approchait, approchaient': 'pemap',
 u'apr\xe8s'

In [19]:
racineChanges={}
for mot in prevLexique:
    if mot in nextLexique:
        if prevLexique[mot] in nomsPropresP:
            racineChanges[nextLexique[mot]]=prevLexique[mot]
        else:
            racineChanges[nextLexique[mot]]=modifierRadical(randomRadical(prevLexique[mot]),dist=[0,0,0,1,1,2])
    else:
        print "***"
        print mot,"n'est pas dans le nouveau lexique"
        print "***"

random rokam
modif rokam
random belef
1 <function apophonie at 0x7f9488957ed8>
modif belof
random nuTuN
2 <function devoiserFinale at 0x7f9488957cf8>
2 <function changerLongueur at 0x7f948885d050>
2 <function apophonie at 0x7f9488957ed8>
2 <function voiserVCV at 0x7f9488957b90>
1 <function changerLongueur at 0x7f948885d050>
1 <function changerLongueur at 0x7f948885d050>
modif nuDuNu
random jaken
modif jaken
random laZaT
2 <function changerLongueur at 0x7f948885d050>
2 <function devoiserFinale at 0x7f9488957cf8>
2 <function apophonie at 0x7f9488957ed8>
1 <function apophonie at 0x7f9488957ed8>
modif luZuT
random kanar
1 <function devoiserFinale at 0x7f9488957cf8>
1 <function voiserVCV at 0x7f9488957b90>
1 <function apophonie at 0x7f9488957ed8>
modif kanur
random tutep
modif tutep
random kufaf
1 <function changerLongueur at 0x7f948885d050>
1 <function devoiserFinale at 0x7f9488957cf8>
1 <function metathese at 0x7f9488957de8>
1 <function changerLongueur at 0x7f948885d050>
1 <function metat

In [20]:
modifierRadical("Tonif")

1 <function devoiserFinale at 0x7f9488957cf8>
1 <function changerLongueur at 0x7f948885d050>
modif Tonifi


'Tonifi'

In [21]:
racineChanges

{'Desop': 'zorus',
 'Dodig': 'nukuS',
 'Dulef': 'donaNo',
 'Dutet': 'kakip',
 'NaNov': 'nuzoT',
 'Nadem': 'DeZiSo',
 'Naj': 'butud',
 'Nasuz': 'fudiD',
 'NeNet': 'kuSit',
 'Nenam': 'tupaS',
 'NiDaN': 'gelom',
 'NipiS': 'tuvom',
 'Nipot': 'duZut',
 'NoNab': 'fejug',
 'Noban': 'sisiSu',
 'Nobop': 'rakij',
 'NuSuN': 'roNoN',
 'Nutuk': 'TiTaS',
 'SeDiT': 'kuvaf',
 'Selut': 'tuvoS',
 'Sijap': 'Tafak',
 'Sik': 'Tom',
 'Sipuz': 'dupod',
 'SodoS': 'bemes',
 'Sokat': 'kilit',
 'Sokuv': 'tedol',
 'SonuS': 'wazar',
 'Soruf': 'varep',
 'Sulowi': 'kikade',
 'Supin': 'fozol',
 'Sutif': 'kokek',
 'TaNam': 'didef',
 'Tamen': 'Sapis',
 'Tapat': 'Zogon',
 'Tebeb': 'Dedok',
 'TegiN': 'zuvod',
 'Tesip': 'vuSun',
 'Tewid': 'lewop',
 'Tik': 'nopa',
 'ToDoS': 'nigef',
 'TomoS': 'fonoS',
 'TotuS': 'SoZas',
 'Tozem': 'zabon',
 'Zag': 'deZ',
 'ZakuD': 'Nitan',
 'Zarut': 'Natav',
 'Zefal': 'ledaf',
 'Zetut': 'pufam',
 'ZiDuN': 'pidop',
 'ZipZir': 'NaftuT',
 'Zipan': 'bigad',
 'ZobeT': 'botuk',
 'Zoj': 'deg',
 'Z

In [22]:
newStems=replace_keys(prevStems,racineChanges)

In [23]:
with open(repertoire+'randomStems.yaml', 'w') as output:
    yaml.dump(newStems, output, default_flow_style=False,allow_unicode=True)

In [24]:
with open(repertoire+"randomStems.yaml", 'r') as input:
    yamlLines=input.readlines()

In [25]:
yamlText="".join(yamlLines)
yamlText=re.sub(r"\n\s*-\s*",", ",yamlText)
yamlText=re.sub(r":,\s*",": ",yamlText)
yamlText=re.sub(r": *([^\n]+)",": [\g<1>]",yamlText)
print yamlText

ADJ:
  A1:
    Dunumu: [blanc, blancs, blanche, blanches]
    NigeZ: [rouge, rouges]
    buziT: [autre, autres]
    doZon: [vilain, vilains, vilaine, vilaines]
    gopek: [vert, verts, verte, vertes]
    joluf: [quatre]
    luruN: [petit, petits, petite, petites]
    mukop: [différent, différents, différente, différentes]
    potoZ: [courageux, courageuse, courageuses]
    ribar: [blessé, blessés, blessée, blessées]
    roZup: [méchant, méchants, méchante, méchantes]
    rokam: [jaune, jaunes]
    romuz: [grand, grands, grande, grandes]
    rukuk: [disparu, disparus, disparue, disparues]
    sisiSu: [troisième, troisièmes]
    zegim: [treize]
    zegip: [nouveau, nouveaux, nouvelle, nouvelles]
  A2:
    Nulov: [cinq]
    TiZiZ: [furieux, furieuse, furieuses]
    Tizan: [terrible, terribles]
    Zogon: [gros, grosse, grosses]
    gelom: [dernier, derniers, dernière, dernières]
    kiTit: [bas, basse, basses]
    kurike: [six]
    luSun: [suivant, suivants, suivante, suivantes]
    pekiT

In [26]:
with open(repertoire+'randomStems.yaml', 'w') as output:
    output.write(yamlText)